# 단어의 표현 (Word Representation)


기계는 문자를 그대로 인식할 수 없기때문에 숫자로 변환



# 1 원-핫 인코딩 (One-Hot Encoding)

##1-1 직접 구현해보기

###"원숭이, 바나나, 사과" 로 원-핫 인코딩을 한다면

In [ ]:
# 인코딩 대상 단어들을 담은 리스트
word_ls = ['원숭이','바나나','사과','개', '고양이']

In [ ]:
from collections import defaultdict
import numpy as np 

def one_hot_encode(word_ls):
    # 고유 단어와 인덱스를 매칭시켜주는 사전 생성
    word2id_dic = defaultdict(lambda:len(word2id_dic)) 
    
    # {단어 : 인덱스} 사전 구축
    for word in word_ls:
        word2id_dic[word]
    
    n_unique_words = len(word2id_dic) # 고유한 단어의 갯수
    one_hot_vectors = np.zeros((len(word_ls), n_unique_words)) # 원핫-벡터를 만들기 위해 비어있는 벡터 생성
    
    for i,word in enumerate(word_ls):
        index = word2id_dic[word] # 해당 단어의 고유 인덱스
        one_hot_vectors[i, index] = 1 # 해당 단어의 고유 인덱스에만 1을 더해줌
        
    return one_hot_vectors

In [ ]:
one_hot_vectors = one_hot_encode(word_ls)
one_hot_vectors

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

###"코끼리"라는 단어가 추가된다면?

In [ ]:
word_ls = ['원숭이','바나나','사과','코끼리']

In [ ]:
one_hot_vectors = one_hot_encode(word_ls)
one_hot_vectors

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

## 1-2 sklearn 활용


함수명 | 설명
--|--
fit(X[, y])	| Fit OneHotEncoder to X.
fit_transform(X[, y])	| Fit OneHotEncoder to X, then transform X.
inverse_transform(X)	| Convert the back data to the original representation.
transform(X)	| Transform X using one-hot encoding.

In [ ]:
# sklearn을 활용한 one-hot encoding
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# 예제 데이터 배열
values = array(word_ls)
print(values)

# 문자열에 숫자를 붙임
label_enc = LabelEncoder()
int_enc = label_enc.fit_transform(values)
print(int_enc)

# binary encode
onehot_enc = OneHotEncoder(sparse=False)
int_enc = int_enc.reshape(len(int_enc), 1) # n:1 matrix로 변환
print(int_enc)
onehot_enc = onehot_enc.fit_transform(int_enc)
print(onehot_enc)

# one-hot encoding 의 첫번째 배열을 값을 역으로 산출
inverted = label_enc.inverse_transform([argmax(onehot_enc[0, :])])
print(inverted)

['원숭이' '바나나' '사과' '코끼리']
[2 0 1 3]
[[2]
 [0]
 [1]
 [3]]
[[0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]
['원숭이']




---



# 2 유사도 계산

## 2-1 유클리디언 거리(Euclidean distance)
두 벡터사이의 직선 거리. 피타고라스 정리를 생각하면 이해하기 쉬움

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/55/Euclidean_distance_2d.svg/220px-Euclidean_distance_2d.svg.png"  width="200"/>

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/795b967db2917cdde7c2da2d1ee327eb673276c0" width="350"/>

https://en.wikipedia.org/wiki/Euclidean_distance

In [ ]:
word_embedding_dic = {
    '사과' : [1.0, 0.5],
    '바나나' : [0.9, 1.2],
    '원숭이' : [0.5, 1.5]
}

In [ ]:
import numpy as np
def euclidean_dist(x,y):   
    x = np.array(x)
    y = np.array(y)
    return np.sqrt(np.sum((x-y)**2))
  
# 사과와 바나나의 코사인 유사도
euclidean_dist(word_embedding_dic['사과'], word_embedding_dic['바나나'])

0.7071067811865475

In [ ]:
np.sqrt(np.power(1.0-0.9,2.0) + np.power(0.5-1.2, 2.0))

0.7071067811865475

## 2-2 자카드 유사도(Jaccard index)

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/1f/Intersection_of_sets_A_and_B.svg/200px-Intersection_of_sets_A_and_B.svg.png" />

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/eaef5aa86949f49e7dc6b9c8c3dd8b233332c9e7" />

https://en.wikipedia.org/wiki/Jaccard_index

In [ ]:
s1 = '대부분 원숭이는 바나나를 좋아합니다.'
s2 = '코주부 원숭이는 바나나를 싫어합니다.'

# 토큰화를 수행합니다.
token_s1 = s1.split()
token_s2 = s2.split()

union = set(token_s1).union(set(token_s2))
print(union)

intersection = set(token_s1).intersection(set(token_s2))
print(intersection)

print(len(intersection)/len(union)) # 2를 6로 나눔.

{'좋아합니다.', '바나나를', '코주부', '대부분', '싫어합니다.', '원숭이는'}
{'원숭이는', '바나나를'}
0.3333333333333333


## 2-3 코사인 유사도(Cosine Similarity) 

*  두 벡터간의 유사도를 측정하는 방법 중 하나
*  두 벡터 사이의 코사인을 측정
*  0도 = 1, 90도 = 0, 180도 = -1   ==> 1에 가까울수록 유사도가 높음




<img src="https://www.oreilly.com/library/view/statistics-for-machine/9781788295758/assets/2b4a7a82-ad4c-4b2a-b808-e423a334de6f.png" width="300"/>

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/1d94e5903f7936d3c131e040ef2c51b473dd071d" width='350'/>

https://en.wikipedia.org/wiki/Cosine_similarity

In [ ]:
def cosine_similarity(x, y):
    # x와 y, 두 벡터의 코사인 유사도를 계산하는 함수
    nominator = np.dot(x, y)    # 분자
    denominator = np.linalg.norm(x)*np.linalg.norm(y)  # 분모
    return nominator/denominator

In [ ]:
# 사과와 바나나의 코사인 유사도
print(cosine_similarity(word_embedding_dic['사과'], word_embedding_dic['바나나']))
print(euclidean_dist(word_embedding_dic['사과'], word_embedding_dic['바나나']))

0.8944271909999159
0.7071067811865475


In [ ]:

# 사과와 원숭이의 코사인 유사도
print(cosine_similarity(word_embedding_dic['사과'], word_embedding_dic['원숭이']))
print(euclidean_dist(word_embedding_dic['사과'], word_embedding_dic['원숭이']))

0.7071067811865475
1.118033988749895


In [ ]:
# 바나나와 원숭이의 코사인 유사도
print(cosine_similarity(word_embedding_dic['바나나'], word_embedding_dic['원숭이']))
print(euclidean_dist(word_embedding_dic['바나나'], word_embedding_dic['원숭이']))

0.9486832980505138
0.5




---

